In [1]:
#Imports
import os
import tensorflow as tf

import sys
sys.path.insert(0, '..')
from utils.federatedAggregation import FederatedAggregation
from utils.reinforcementLearningHelper import *
from utils.federatedLearningHelper import *

In [2]:
#Setup Environments of selected buildings for training, evaluation, and testing

environments, observation_spec, action_spec  = setup_energymanagement_environments(num_buildings=30)

#Check environment setup
print(
    "Batch size:", environments["train"][f"building_1"].batch_size, 
    "/ State Space: {} / Action Space: {}".format(observation_spec.shape[0], action_spec.shape[0]),
    "/ Upper bound: {}".format(round(environments["train"][f"building_1"].action_spec().maximum.item(), 3)),
)

Batch size: 1 / State Space: 6 / Action Space: 1 / Upper bound: 2.3


In [3]:
# Cluster similar buildings (K-Means with DTW)

#Set parameter
num_clusters = 10 # 2, 6, 10, 12, 14, 16, 18

clustered_buildings = load_clustered_buildings(num_clusters)
clustered_buildings

{0: array([7], dtype=int64),
 1: array([16, 17, 21], dtype=int64),
 2: array([13, 19, 20], dtype=int64),
 3: array([18], dtype=int64),
 4: array([ 3,  4,  9, 14, 15, 22, 30], dtype=int64),
 5: array([1], dtype=int64),
 6: array([6, 8], dtype=int64),
 7: array([11], dtype=int64),
 8: array([ 5, 12, 23, 24, 25, 26, 27, 28, 29], dtype=int64),
 9: array([ 2, 10], dtype=int64)}

In [4]:
# Setup Agent networks
federated_rounds = 3
batch_size = 128
replay_buffer_capacity = 20000 #-> only <18.000 samples per dataset
initial_collect_steps = 200 #2000
collect_steps_per_iteration = 20 
num_iterations = 100 #10000
eval_interval = 95 #9999

In [5]:
# FEDERATED LEARNING - Initalization Round 0

tf.compat.v1.reset_default_graph()
global_step = tf.compat.v1.train.get_or_create_global_step()

#Initalize a global model for each Cluster of similar buildings
for cluster in range(num_clusters):
        
        # 1. Build global agent per cluster
        first_building_in_cluster = clustered_buildings[cluster][0]
        global_ppo_agent, global_eval_policy, global_collect_policy = initialize_ppo_agent(
                observation_spec=observation_spec, action_spec=action_spec, global_step=global_step, environments=environments,
                )

        # 2. Initially store weights
        model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster}/FLround{0}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        
        save_ppo_weights(global_ppo_agent, model_dir)

In [6]:
#FEDERATED LEARNING - Model training for multiple Rounds

#For each federated round and cluster
for federated_round  in range(federated_rounds):
    for cluster_number, buildings_in_cluster in clustered_buildings.items():

        #Iterate through the buildings per cluster
        print(f"Cluster {cluster_number}: Buildings {buildings_in_cluster} Federated round ---", federated_round+1, f"/ {federated_rounds}")
        local_storage = {
            "actor_weights": [], "value_weights": [], "performance_metrics": []
            }
        
        for building_index in buildings_in_cluster:
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()
            
            #1. Initalize local agent
            local_ppo_agent, local_eval_policy, local_collect_policy = initialize_ppo_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster_number}/FLround{federated_round}_c{num_clusters}_AvgAgg")
            local_ppo_agent = set_weights_to_ppo_agent(local_ppo_agent, model_dir)
            
            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            local_iterator, local_collect_driver, local_time_step, local_policy_state = setup_rl_training_pipeline(
                local_ppo_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                local_collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Train, evaluate agent and store weights
            local_ppo_agent, local_storage = local_agent_training_and_evaluation(
                local_iterator, local_collect_driver, local_time_step, local_policy_state, global_step, 
                local_ppo_agent, local_eval_policy, local_storage, building_index, num_iterations, environments, agent_type="ppo"
                )           

        # Performe Federated aggregation
        average_actor_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["actor_weights"], local_storage["performance_metrics"])
        average_value_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["value_weights"], local_storage["performance_metrics"]) 
        
        #Save federated weights for next round (Round + 1)
        model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster_number}/FLround{federated_round+1}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        save_federated_ppo_weights(model_dir, average_actor_weights, average_value_weights)

Cluster 0: Buildings [7] Federated round --- 1 / 3
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Return:  -74047.695
Cluster 1: Buildings [16 17 21] Federated round --- 1 / 3
Return:  -37190.617
Return:  -704.67554
Return:  -51346.832
Cluster 2: Buildings [13 19 20] Federated round --- 1 / 3
Return:  -84010.32
Return:  -82364.055
Return:  -84678.09
Cluster 3: Buildings [18] Federated round --- 1 / 3
Return:  -47902.453
Cluster 4: Buildings [ 3  4  9 14 15 22 30] Federated round --- 1 / 3
Return:  -88413.81
Return:  -88689.38
Return:  -86646.984
Return:  -90582.08
Return:  -92183.29
Return:  -88065.57
Return:  -88133.18
Cluster 5: Buildings [1] Federated round --- 1 / 3
Return:  -838.1006
Cluster 6: Buildings [6 8] Federated round --- 1 / 3
Return:  -45855.32
Return:  -70924.55
Cluster 7: Buildings [11] Federated round --- 1 / 3
Return:  -22786.838
Cluster 8: Buildings [ 5 12 23 24 25 26 27 28 29] Federated round --- 1 / 3
Return:  -120049.664

In [7]:
# LOCAL REFITTING AND EVALUATION

best_federated_round = 3
num_rounds=1
num_test_iterations = 200

result_df = pd.DataFrame(columns=['Building', 'Total Profit'])

for cluster_number, buildings_in_cluster in clustered_buildings.items():
    for building_index in buildings_in_cluster:
        
        for round in range(num_rounds):
            print("Cluster: ", cluster_number, " - Building: ", building_index, " - round: ", round)
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()

            #1. Initalize local agent
            tf_ppo_agent, eval_policy, collect_policy = initialize_ppo_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster_number}/FLround{best_federated_round}_c{num_clusters}_AvgAgg")
            tf_agent = set_weights_to_ppo_agent(tf_ppo_agent, model_dir)

            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            iterator, collect_driver, time_step, policy_state = setup_rl_training_pipeline(
                tf_ppo_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Setup wandb logging
            artifact = initialize_wandb_logging(name=f"Exp_building{building_index}_rd{round}", num_iterations=num_iterations)
            
            #5. Train, evaluate agent and store weights
            result_df, metrics = agent_training_and_evaluation(global_step, num_test_iterations, collect_driver, 
                time_step, policy_state, iterator, tf_ppo_agent, eval_policy, building_index, result_df, eval_interval, environments)
            
            #6. End and log wandb
            end_and_log_wandb(metrics, artifact)

Cluster:  0  - Building:  7  - round:  0


c:\Users\rs1044\Documents\GitHub\Federated-Reinforcement-Learning-for-Battery-Charging\src\notebooks\..\utils\reinforcementLearningHelper.py:338: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({'Building': [building_index], 'Total Profit': [wandb.summary["Final Profit"]]})], ignore_index=True)


Building:  7  - Total Profit:  -32.98025289999993
Cluster:  1  - Building:  16  - round:  0
Building:  16  - Total Profit:  -161.0366269000002
Cluster:  1  - Building:  17  - round:  0
Building:  17  - Total Profit:  -368.11111181999956
Cluster:  1  - Building:  21  - round:  0
Building:  21  - Total Profit:  -412.2055835700018
Cluster:  2  - Building:  13  - round:  0


KeyboardInterrupt: 